In [1]:
import pandas as pd

df = pd.read_csv('data.csv')
df

label                                             review
0        0                          味道一般，酸辣土豆丝酸的没法吃。。。肝尖也做的一般
1        0  订的菜没有了，换了个便宜的菜，说了多退少补，结果没把钱给我退回来。诚信很有问题。锅包肉又硬又...
2        0                                       太慢,太慢,太慢,不好吃
3        0                                 商家距离近，但速度太慢，饼子都凉了。
4        0  久久鸭的口味偏颇，辣味不是很浓。总体而言还行。但是没有餐具，虽然我的单里说明要餐具了，这是一...
..     ...                                                ...
195      0                                      两个小时才送到,我是真苦了
196      1         晚了半小时左右吧……不过最近这么冷还是辛苦了，估计大家都不想出门吧。超级好吃！推荐！
197      0                        没吃过这么难吃的水煮牛肉，刚送来吃一口就扔了。有股膻味
198      0                                        送餐慢，肉少，味道不好
199      0                             下错单了,不过冷面味道不错,但是没有给我小菜

[200 rows x 2 columns]

In [14]:
from openai import OpenAI
from pydantic import BaseModel
from typing import List
import os
import instructor



#结构化输出
class UserDetail(BaseModel):
    name: str
    age: int
    hobby: List[str]
    

#Prompt提示
PROMPT_TEXT = "根据自我介绍文本内容，从中提取出姓名、年龄、兴趣"

#实验数据
introduction_text = '我是张三，今年34岁， 来自黑龙江省， 我的兴趣爱好有打篮球、踢足球、游泳、打游戏。'


client = instructor.from_openai(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="NA",  # required, but unused
    ),
    mode = instructor.Mode.JSON,
)


resp = client.chat.completions.create(
    model = "qwen2.5:7b",
    messages=[
        {"role": "system", "content": PROMPT_TEXT},
        {"role": "user", "content": introduction_text}
    ],
    response_model = UserDetail,
    max_retries = 3
)


print(resp.model_dump_json(indent=2))


{
  "name": "张三",
  "age": 34,
  "hobby": [
    "打篮球",
    "踢足球",
    "游泳",
    "打游戏"
  ]
}


In [27]:
%%time
from openai import OpenAI
from pydantic import BaseModel
from typing import List
import os
import instructor

#结构化输出
class Sentiment(BaseModel):
    senti_label: str
    senti_score: float


#Prompt提示
PROMPT_TEXT = "根据评论内容，返回文本的情感类别(pos、neg、neo)和对应的情感得分(取值范围0~1)"

#实验数据
COMMENT_CONTENT = '11点14订餐，13点20饭才到，2个小时才把我的午饭送到，而且还是打了2次客服电话，1次投诉电话才给送来，要是不打电话都不知道几点能吃上午饭？'


client = instructor.from_openai(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="NA",  # required, but unused
    ),
    mode = instructor.Mode.JSON,
)


resp = client.chat.completions.create(
    model = "qwen2.5:3b",
    messages=[
        {"role": "system", "content": PROMPT_TEXT}, #提示PROMP
        {"role": "user", "content": COMMENT_CONTENT} #评论文本
    ],
    response_model = Sentiment,
    max_retries = 3
)


print(resp.model_dump_json(indent=2))

{
  "senti_label": "neg",
  "senti_score": 0.1
}
CPU times: user 37.5 ms, sys: 4.47 ms, total: 42 ms
Wall time: 577 ms


In [10]:
%%time
import pandas as pd
from tqdm import tqdm
from openai import OpenAI
from pydantic import BaseModel
from typing import List
import numpy as np
import os
import instructor

 

#结构化输出
class Sentiment(BaseModel):
    senti_label: str
    senti_score: float
    

#Prompt提示
PROMPT_TEXT = "根据评论内容，返回文本的情感类别(pos、neg)和情感得分(取值范围 -1~1)" 

client = instructor.from_openai(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="NA",  # required, but unused
    ),
    mode = instructor.Mode.JSON,
)


labels = []
scores = []

#读取数据
df = pd.read_csv('data.csv')
for review in tqdm(df['review']):
    try:
        resp = client.chat.completions.create(
        model = 'deepseek-r1:1.5b',  #选择模型。 3b、7b等
        messages=[
            {"role": "system", "content": PROMPT_TEXT},  #提示
            {"role": "user", "content": review}   #评论文本
        ],
        response_model = Sentiment,
        max_retries = 3  #最大(失败）的重试次数。 
    )
        
        labels.append(resp.senti_label)
        scores.append(resp.senti_score)
    except:
        labels.append('NA')
        scores.append(np.nan)
        
    
df['SentiLabel'] = labels
df['SentiScore'] = scores
#保存结果
df.to_csv('deepseekr1-1.5b-result.csv', index=False)
df

100%|█████████████████████████████████████████| 200/200 [05:47<00:00,  1.74s/it]

CPU times: user 6.06 s, sys: 413 ms, total: 6.47 s
Wall time: 5min 47s


label                                             review SentiLabel  \
0        0                          味道一般，酸辣土豆丝酸的没法吃。。。肝尖也做的一般         NA   
1        0  订的菜没有了，换了个便宜的菜，说了多退少补，结果没把钱给我退回来。诚信很有问题。锅包肉又硬又...         NA   
2        0                                       太慢,太慢,太慢,不好吃        不好吃   
3        0                                 商家距离近，但速度太慢，饼子都凉了。         NA   
4        0  久久鸭的口味偏颇，辣味不是很浓。总体而言还行。但是没有餐具，虽然我的单里说明要餐具了，这是一...         NA   
..     ...                                                ...        ...   
195      0                                      两个小时才送到,我是真苦了         NA   
196      1         晚了半小时左右吧……不过最近这么冷还是辛苦了，估计大家都不想出门吧。超级好吃！推荐！         NA   
197      0                        没吃过这么难吃的水煮牛肉，刚送来吃一口就扔了。有股膻味         NA   
198      0                                        送餐慢，肉少，味道不好         NA   
199      0                             下错单了,不过冷面味道不错,但是没有给我小菜         NA   

     SentiScore  
0           NaN  
1           NaN  
2          -1.0  
3           NaN  
4           NaN  
..          ...  
195         NaN  
196         NaN  
197         NaN  
198         NaN  
199         NaN  

[200 rows x 4 columns]

In [5]:
df['sentiment'].value_counts()

sentiment
neg    128
pos     69
NA       3
Name: count, dtype: int64

In [ ]:
expression = "(label == 1) & (SentiLabel == 'pos') | (label == 0) & (SentiLabel == 'neg')"
correct_ratio = len(df.query(expression))/ len(df)

print(f'准确率: {correct_ratio*100}%')